# Voronoi Cell Area Analysis

Load packages and distance functions

In [ ]:
import networkx as nx
import itertools as itrtls
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

def distance(x1,y1,x2,y2):
    return ((x2-x1)**2+(y2-y1)**2)**(1/2)

def getDist(x1,y1,x2,y2,xShift=1,yShift=1):
    dists = []
    dists.append(distance(x1,y1,x2,y2))
    dists.append(distance(x1,y1,x2-xShift,y2))
    dists.append(distance(x1,y1,x2+xShift,y2))
    dists.append(distance(x1,y1,x2,y2-yShift))
    dists.append(distance(x1,y1,x2,y2+yShift))
    dists.append(distance(x1,y1,x2-xShift,y2-yShift))
    dists.append(distance(x1,y1,x2-xShift,y2+yShift))
    dists.append(distance(x1,y1,x2+xShift,y2-yShift))
    dists.append(distance(x1,y1,x2+xShift,y2+yShift))
    return min(dists),dists.index(min(dists))

def getPointFromIndex(x,y,minInd,xShift=1,yShift=1):
    if(minInd == 0):
        return x,y
    elif(minInd == 1):
        return x-xShift, y
    elif(minInd == 2):
        return x+xShift, y
    elif(minInd == 3):
        return x, y-yShift
    elif(minInd == 4):
        return x, y+yShift
    elif(minInd == 5):
        return x-xShift, y-yShift
    elif(minInd == 6):
        return x-xShift, y+yShift
    elif(minInd == 7):
        return x+xShift, y-yShift
    else:
        return x+xShift, y+yShift

Load voronoi network<br>
Possible netNames: HIP, Poisson, RSA, Gaussian, Stealthy0.20, Stealthy0.40, Stealthy0.49<br>
The hyperuniform point patterns are built on a 100x100 box and the nonhyperuniform point patterns are built on a 1x1 box, so we scale the hyperuniform point patterns down to have all systems on the same scale.

In [2]:
netName = "HIP"
num = "01"
G = nx.read_edgelist("Network/"+netName+'_'+num+'_voronoi_edgelist.txt', nodetype = int, data = [('distance',float)])
if(netName in ["Gaussian","Stealthy0.20","Stealthy0.40","Stealthy0.49"]):
    for e in G.edges():
        G.edges()[e]['distance'] /= 100
G = G.subgraph(max(nx.connected_components(G), key = len))
nodes = list(G.nodes())

Load point pattern

In [6]:
f = open("Network/"+netName+'-'+num+'_voronoi.txt','r')
lines = f.readlines()
f.close()
coords = [x.rstrip().split('\t') for x in lines if x.rstrip().split(' ') != ['']]
coords = [[float(x[0]),float(x[1])] for x in coords]
coords = np.array(coords)
if(netName in ["Gaussian","Stealthy0.20","Stealthy0.40","Stealthy0.49"]):
    coords /= 100

pointDict = {}
for n in nodes:    
    pointDict[n] = (float(coords[n-1][0]),float(coords[n-1][1]))

In [7]:
len(coords)

20000

Find faces of network

In [ ]:
allFaces = []
pct = 1
print('0%')
for loc,n in enumerate(G.nodes()):
    if(loc/len(G)*100>pct):
        print(str(pct)+"%")
        pct += 1
    combs = list(itrtls.combinations(G.neighbors(n),2))
    cliques = list(nx.find_cliques(G,[n]))
    for clique in cliques:
        if(len(clique) == 4):
            edgeNodes = []
            centers = {}
            for node in clique:
                cs = list(nx.find_cliques(G,[node]))
                all4 = True
                num4 = 0
                for c in cs:
                    if(len(c) != 4):
                        all4 = False
                    else:
                        num4 += 1
                if(all4):
                    centers[node] = num4
            center = list(centers)[0]
            for c in centers:
                if(centers[c] < centers[center]):
                    center = c
            clique.remove(center)
            for comb in itrtls.combinations(clique,2):
                if(tuple(sorted(comb)) in combs):
                    combs.remove(tuple(sorted(comb)))
    G2 = G.copy()
    for neigh in G.neighbors(n):
        G2.remove_edge(n,neigh)
    faces = []
    for _ in range(G.degree(n)):
        cycles = []
        for c in combs:
            if(nx.has_path(G2, c[0], c[1])):
                cycles.append(nx.shortest_path(G2, c[0],c[1]))
        if(len(cycles) == 0):
            continue
        cycleLens = np.array([len(c) for c in cycles])
        if(len(np.where(cycleLens == min(cycleLens))[0]) > 1):
            allSmallest = []
            smallestLens = []
            for c in cycles:
                if(len(c) == min(cycleLens)):
                    allSmallest.append(c)
                    cycleLen = G.edges()[(n,c[0])]['distance']
                    for i in range(len(c)-1):
                        e1 = c[i]
                        e2 = c[(i+1)]
                        cycleLen += G.edges()[(e1,e2)]['distance']
                    cycleLen += G.edges()[(c[-1],n)]['distance']
                    smallestLens.append(cycleLen)
            smallest = allSmallest[smallestLens.index(min(smallestLens))]
        else:  
            smallest = min(cycles,key=len)
        combs.remove((smallest[0],smallest[-1]))
        faces.append([n]+smallest)
        for i in range(len(smallest)-1):
            G2.remove_edge(smallest[i],smallest[(i+1)])
            if(i > 0):
                G2.remove_node(smallest[i])
    for f in faces:
        minLoc = f.index(min(f))
        if(f[(minLoc+1)%len(f)] < f[(minLoc-1)%len(f)]):
            f2 = f[minLoc:]+f[:minLoc]
        else:
            f2 = f[minLoc::-1]+f[:minLoc:-1]
        if(f2 not in allFaces):
            allFaces.append(f2)
print('100%')

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96


Find areas of faces

In [ ]:
faceAreas = {}
for f in allFaces:
    area = 0
    originalX,originalY = pointDict[f[0]]
    originalPoint = f[0]
    for f_i in range(len(f)):
        n1 = f[f_i]
        n2 = f[(f_i+1)%len(f)]
        x1,y1 = pointDict[n1]
        possibleCoords = [(x1,y1),(x1-1,y1),(x1+1,y1),(x1,y1-1),(x1,y1+1),(x1-1,y1-1),(x1-1,y1+1),(x1+1,y1-1),(x1+1,y1+1)]
        for i in range(len(possibleCoords)):
            xTemp,yTemp = possibleCoords[i]
            dist = distance(originalX,originalY,xTemp,yTemp)
            netDist = nx.shortest_path_length(G,originalPoint,n1,weight = 'distance')
            if(np.isclose(dist,netDist) or  dist <= netDist):
                x1,y1 = possibleCoords[i]
        x2,y2 = pointDict[n2]
        possibleCoords = [(x2,y2),(x2-1,y2),(x2+1,y2),(x2,y2-1),(x2,y2+1),(x2-1,y2-1),(x2-1,y2+1),(x2+1,y2-1),(x2+1,y2+1)]
        for i in range(len(possibleCoords)):
            xTemp,yTemp = possibleCoords[i]
            dist = distance(originalX,originalY,xTemp,yTemp)
            netDist = nx.shortest_path_length(G,originalPoint,n2,weight = 'distance')
            if(np.isclose(dist,netDist) or  dist <= netDist):
                x2,y2 = possibleCoords[i]
        
        area += (x1*y2-y1*x2)
        #print(area)
    area = abs(area/2)
    faceAreas[str(f)] = area
faceAreas = pd.DataFrame(faceAreas.values(),index = faceAreas.keys())

Find centers of faces

In [ ]:
faceDict = {f:faceAreas.loc[f,0] for f in faceAreas.index}
faceCenterXs = {}
faceCenterYs = {}
for f in faceAreas.index:
    points = [int(x) for x in f[1:-1].split(", ")]
    firstPoint = pointDict[points[0]]
    pos = [firstPoint]
    for p in points[1:]:
        c = pointDict[p]
        dist,minInd = getDist(firstPoint[0], firstPoint[1], c[0], c[1])
        pos.append(getPointFromIndex(c[0],c[1],minInd))
    centerX,centerY = np.average(np.array(pos),axis=0)
    faceCenterXs[f] = centerX
    faceCenterYs[f] = centerY
xDF = pd.DataFrame(list(faceCenterXs.values()),index=list(faceCenterXs.keys()),columns=["X"])
yDF = pd.DataFrame(list(faceCenterYs.values()),index=list(faceCenterYs.keys()),columns=["Y"])
faces = pd.concat([faceAreas,xDF,yDF],axis=1)

Find distances between faces

In [ ]:
faceDists = np.zeros((len(faces),len(faces)))
pct = 1
for i in range(len(faces)):
    if(i/len(faces)*100 > pct):
        print(pct)
        pct += 1
    x1 = faces.iloc[i]["X"]
    y1 = faces.iloc[i]["Y"]
    for j in range(i):
        x2 = faces.iloc[j]["X"]
        y2 = faces.iloc[j]["Y"]
        
        dist = getDist(x1, y1, x2, y2)[0]
        faceDists[i][j] = dist
        faceDists[j][i] = dist

Find face area correlation function

In [ ]:
C00 = []
rs = np.arange(0,0.2,0.001)
for i in range(1,len(rs)):
    print((i*100)//len(rs))
    A = []
    ASq = []            
    faceDistBins = np.digitize(faceDists,rs,right=True)   
    locs = np.where(faceDistBins == i)
    for j in range(len(locs[0])):
        A1 = faceAreas.iloc[locs[0][j]][0]
        A2 = faceAreas.iloc[locs[1][j]][0]
        A.append(A1)
        A.append(A2)
        ASq.append(A1*A2)
avgA = np.average(A)
varA = np.var(A)
avgASq = np.average(ASq)
C00.append((avgASq-avgA**2)/varA)
            
df = pd.DataFrame(C00,index=rs[1:],columns=["C00"])
df["Moving Average"] = df["C00"].rolling(window=5).mean()
plt.figure(0,dpi=200)
plt.plot(df.index,df["C00"],label=netDict[netNum],alpha=0.5)
plt.figure(1,dpi=200)
plt.plot(df.index,df["Moving Average"],label=netDict[netNum],alpha=0.5)
plt.figure(0,dpi=200)
plt.xlabel("R")
plt.ylabel("Area Correlation Function")
plt.legend(bbox_to_anchor=(1,1))
plt.figure(1,dpi=200)
plt.xlabel("R")
plt.ylabel("Area Correlation Function - Moving Average")
plt.legend(bbox_to_anchor=(1,1))